In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy import optimize
from matplotlib import cm
from math import isinf
from scipy.optimize import linprog

In [2]:
class JointValuationEnv:
    def __init__(self, mean, cov, costs, N, seed=None):
        self.mean = np.array(mean)
        self.cov = np.array(cov)
        self.costs = np.array(costs)
        self.N = N
        self.rng = np.random.default_rng(seed)

    def sample_valuation(self):
        while True:
            val = self.rng.multivariate_normal(self.mean, self.cov)
            if np.all((val >= 0) & (val <= 1)):
                return val

    def round(self, offered_products, prices):
        valuations = self.sample_valuation()
        purchases = np.zeros(self.N, dtype=int)
        revenue = 0.0
        for i in offered_products:
            if valuations[i] >= prices[i]:
                purchases[i] = 1
                revenue += prices[i] - self.costs[i]
        return purchases, revenue

In [3]:
class UCBPricingAgentWithBudget:
    def __init__(self, n_products, n_prices, B, T, reward_range=1, cost_range=1):
        self.n_products = n_products
        self.n_prices = n_prices
        self.K = n_products * n_prices
        self.B = B
        self.T = T
        self.budget = B
        self.rho = B / T  # budget per round
        self.t = 0

        self.reward_range = reward_range
        self.cost_range = cost_range

        self.avg_reward = np.zeros(self.K)
        self.avg_cost = np.zeros(self.K)
        self.N_pulls = np.zeros(self.K)

    def compute_ucbs(self):
        # Large value for unexplored arms
        large_value = (1 + np.sqrt(2 * np.log(max(self.T,1)) / 1)) * max(self.reward_range, self.cost_range) * 10
        
        f_ucbs = np.zeros(self.K)
        c_lcbs = np.zeros(self.K)
        
        unexplored = self.N_pulls == 0
        f_ucbs[unexplored] = large_value
        c_lcbs[unexplored] = 0  # optimistic cost lower bound
        
        explored = self.N_pulls > 0
        f_ucbs[explored] = self.avg_reward[explored] + self.reward_range * np.sqrt(2 * np.log(max(self.T,1)) / self.N_pulls[explored])
        c_lcbs[explored] = self.avg_cost[explored] - self.cost_range * np.sqrt(2 * np.log(max(self.T,1)) / self.N_pulls[explored])
        c_lcbs = np.clip(c_lcbs, 0, None)  # cost lower bound can't be negative
        
        return f_ucbs, c_lcbs

    def compute_opt(self, f_ucbs, c_lcbs):
        # LP: maximize sum(f_ucbs * gamma), s.t. sum(c_lcbs * gamma) <= rho, sum(gamma) = 1, gamma in [0,1]
        if np.all(c_lcbs <= 0):
            gamma = np.zeros(len(f_ucbs))
            gamma[np.argmax(f_ucbs)] = 1.0
            return gamma

        c = -f_ucbs  # minimize negative reward = maximize reward
        A_ub = [c_lcbs]
        b_ub = [self.rho]
        A_eq = [np.ones(self.K)]
        b_eq = [1.0]

        res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,
                      bounds=[(0,1)]*self.K, method='highs')

        if not res.success:
            gamma = np.ones(self.K) / self.K
        else:
            gamma = np.maximum(0, res.x)
            s = gamma.sum()
            gamma = gamma / s if s > 0 else np.ones(self.K) / self.K
        return gamma

    def pull_arm(self):
        if self.budget < 1:
            return None  # stop pulling
        
        f_ucbs, c_lcbs = self.compute_ucbs()
        gamma = self.compute_opt(f_ucbs, c_lcbs)

        # Sample arms from gamma distribution until you get one price for each product
        chosen_price_indices = np.full(self.n_products, -1, dtype=int)
        chosen_arms = []

        # Convert flat indices to (product, price)
        # We do repeated sampling with rejection to ensure one price per product
        max_trials = 1000
        for _ in range(max_trials):
            sampled = np.random.choice(self.K, size=self.n_products, p=gamma)
            products = sampled // self.n_prices
            prices = sampled % self.n_prices
            if len(set(products)) == self.n_products:
                chosen_price_indices = prices
                chosen_arms = list(zip(range(self.n_products), prices))
                break
        else:
            # fallback: choose best price per product greedily
            chosen_price_indices = np.argmax(f_ucbs.reshape(self.n_products, self.n_prices), axis=1)
            chosen_arms = list(zip(range(self.n_products), chosen_price_indices))

        self.last_chosen_arms = chosen_arms
        return chosen_arms

    def update(self, rewards, costs):
        # rewards and costs: arrays of shape (n_products,)
        for (prod, price_idx), r, c in zip(self.last_chosen_arms, rewards, costs):
            arm_idx = prod * self.n_prices + price_idx
            self.N_pulls[arm_idx] += 1
            n = self.N_pulls[arm_idx]
            self.avg_reward[arm_idx] += (r - self.avg_reward[arm_idx]) / n
            self.avg_cost[arm_idx] += (c - self.avg_cost[arm_idx]) / n
            self.budget -= c
        self.t += 1

In [12]:
def compute_clairvoyant_multi(products_prices, win_probabilities, cost=0.0, rho=1.0):
    """
    products_prices: shape (n_products, n_prices) array of prices
    win_probabilities: shape (n_products, n_prices) array of win probabilities
    cost: scalar cost per product (assumed same across all for simplicity)
    rho: budget constraint on total expected "cost" (e.g. expected sales)

    Returns:
        gamma: optimal distribution over all (product, price) pairs, shape (n_products, n_prices)
        max_expected_reward: the max expected reward
    """

    n_products, n_prices = products_prices.shape
    n_arms = n_products * n_prices

    prices_flat = products_prices.flatten()
    win_prob_flat = win_probabilities.flatten()

    # Objective coefficients (negative for linprog minimization)
    c = - (prices_flat - cost) * win_prob_flat

    # Constraint 1: sum of expected "costs" (win probabilities) * gamma <= rho
    A_ub = [win_prob_flat]

    b_ub = [rho]

    # Constraint 2: For each product, sum of gamma over all prices = 1 (pick exactly one price per product)
    A_eq = np.zeros((n_products, n_arms))
    for p in range(n_products):
        A_eq[p, p*n_prices:(p+1)*n_prices] = 1
    b_eq = np.ones(n_products)

    bounds = [(0, 1) for _ in range(n_arms)]

    res = optimize.linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

    if not res.success:
        raise RuntimeError("Linear program did not converge")

    gamma = res.x.reshape((n_products, n_prices))
    max_expected_reward = -res.fun

    return gamma, max_expected_reward


In [17]:
import numpy as np
from scipy import stats
from scipy.optimize import linprog
from tqdm import trange

# Assuming your JointValuationEnv and UCBPricingAgentWithBudget classes are defined

# Environment params
n_products = 3
T = 500#12000
B = 200#6500
rho = B / T
epsilon = T ** (-0.33)
K = int(1 / epsilon)
prices = np.linspace(0, 1, K)

n_epochs = 3

# Define means and stddevs for each product's valuation distribution
means = np.array([0.4, 0.5, 0.6])
scales = np.array([0.1, 0.1, 0.1])

all_regrets = []
all_rewards = []
all_pulls = []

for epoch in range(n_epochs):
    # Compute win probabilities (1 - CDF) per product and price
    win_probabilities = np.zeros((n_products, K))
    for p in range(n_products):
        valuation = stats.norm(loc=means[p], scale=scales[p])
        win_probabilities[p] = 1 - valuation.cdf(prices)

    # Compute clairvoyant LP solution and expected utility per round
    # You will need to define compute_clairvoyant_multi to handle multiple products and prices
    gamma_opt, expected_clairvoyant_utility = compute_clairvoyant_multi(np.tile(prices, (n_products, 1)), win_probabilities=win_probabilities, rho=rho)

    # Initialize environment & agent
    env = JointValuationEnv(means, np.diag(scales**2), costs=np.zeros(n_products), N=n_products, seed=epoch)
    agent = UCBPricingAgentWithBudget(n_products, K, B, T, reward_range=1, cost_range=1)

    rewards_history = []
    regrets = []
    pull_counts = np.zeros((n_products, K), dtype=int)

    for t in trange(T, desc=f"Epoch {epoch+1}/{n_epochs}", unit="round"):
        chosen_arms = agent.pull_arm()
        if chosen_arms is None:
            # Budget exhausted
            rewards_history.append(0)
            regrets.append(expected_clairvoyant_utility)
            continue

        # Prepare prices vector for environment call
        chosen_products = [prod for prod, price_idx in chosen_arms]
        chosen_price_indices = [price_idx for prod, price_idx in chosen_arms]
        offered_prices = np.zeros(n_products)
        offered_prices[chosen_products] = prices[chosen_price_indices]

        # Get purchases and revenue from env
        purchases, revenue = env.round(chosen_products, offered_prices)

        # Prepare reward and cost arrays for update (cost=0)
        rewards_arr = np.zeros(n_products)
        costs_arr = np.zeros(n_products)

        for prod, price_idx in chosen_arms:
            rewards_arr[prod] = prices[price_idx] if purchases[prod] == 1 else 0
            costs_arr[prod] = 0  # costs ignored per your request
            pull_counts[prod, price_idx] += 1

        agent.update(rewards_arr, costs_arr)

        rewards_history.append(revenue)
        regrets.append(expected_clairvoyant_utility - revenue)

    all_regrets.append(np.cumsum(regrets))
    all_rewards.append(np.cumsum(rewards_history))
    all_pulls.append(pull_counts)

    print(f"Epoch {epoch + 1} done: Total reward = {sum(rewards_history):.2f}")

# Compute average regret and rewards across epochs
avg_regret = np.mean(all_regrets, axis=0)
std_regret = np.std(all_regrets, axis=0)
avg_rewards = np.mean(all_rewards, axis=0)

# For plots
avg_payments = avg_rewards
std_payments = np.std(all_rewards, axis=0)
avg_pulls = np.mean(all_pulls, axis=0)
std_pulls = np.std(all_pulls, axis=0)


Epoch 1/1: 100%|██████████| 500/500 [00:36<00:00, 13.88round/s]

Epoch 1 done: Total reward = 278.67


In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(10, 15), constrained_layout=True)

# 1) Chosen Prices per Product (pull counts)
for p in range(n_products):
    axs[0].plot(prices, avg_pulls[p], label=f'Product {p+1}')
    axs[0].fill_between(prices, avg_pulls[p] - std_pulls[p], avg_pulls[p] + std_pulls[p], alpha=0.3)
axs[0].set_xlabel('Price')
axs[0].set_ylabel('Number of pulls')
axs[0].set_title('Chosen Prices per Product')
axs[0].legend()

# 2) Cumulative Payments
axs[1].plot(np.arange(T), avg_payments, label='Average Payments')
axs[1].fill_between(np.arange(T), avg_payments - std_payments, avg_payments + std_payments, alpha=0.3)
axs[1].axhline(B, color='red', linestyle='--', label='Budget')
axs[1].set_xlabel('Round $t$')
axs[1].set_ylabel('Cumulative Payments')
axs[1].set_title('Cumulative Payments of UCBPricingAgentWithBudget')
axs[1].legend()

# 3) Cumulative Regret
axs[2].plot(np.arange(T), avg_regret, label='Average Regret')
axs[2].fill_between(np.arange(T), avg_regret - std_regret, avg_regret + std_regret, alpha=0.3)
axs[2].set_xlabel('Round $t$')
axs[2].set_ylabel('Cumulative Regret')
axs[2].set_title('Cumulative Regret of UCBPricingAgentWithBudget')
axs[2].legend()

plt.show()
